In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy
from RNTK_NEW import RNTK

In [3]:
Ti = 2
TiPrime = 2

In [4]:
dataset = "trains"

In [5]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [6]:
N = int(dic["n_patrons1="])
length = int(dic["n_entradas="])

In [7]:
rntk = RNTK(dic)

In [8]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

## LAYER ZERO ATTEMPT


In [9]:
switch_flag = 1
dim_1_i = dim_1 = 2
dim_2 = 2
dim_2_i = 0
dim_num = dim_1 + dim_2 + 1

In [10]:
tiprimes = []
tis = []

print(dim_1_i, dim_2_i, switch_flag)
for d in range(0,dim_num):
    tiprime = dim_1_i
    ti = dim_2_i

    # diag_func(tiprime, ti, dim_1, dim_2, rntk)
    tiprimes.append(tiprime)
    tis.append(ti)

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

2 0 1


## lets make a scan that does a diagonal

In [11]:
from RNTK_NEW import RNTK, diag_func_wrapper, arrays_to_diag

In [12]:
array_of_diags=T.concatenate([diag_func_wrapper(rntk, 2,2,i[0],i[1],n) for i in zip(tiprimes, tis)])

In [13]:
full_lambda, full_phi = arrays_to_diag(array_of_diags, dim_1, dim_2)

In [14]:

print("LAMBDA")
print(full_lambda)
print("PHI")
print(full_phi)

LAMBDA
[[Op(name=_rewriting_take_18, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_22, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_26, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]
 [Op(name=_rewriting_take_30, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_34, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_38, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]
 [Op(name=_rewriting_take_42, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_46, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewriting_take_50, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)]]
PHI
[[Op(name=_rewriting_take_20, fn=_rewriting_take, shape=(10, 10), dtype=float32, scope=/default)
  Op(name=_rewr